# 경사하강법 행렬분해를 통한 잠재요인 협업 필터링 예시
사용자 4명이 5개의 영화에 대한 평점을 넣은 행렬을 R이라고 하자.
||영화1|영화2|영화3|영화4|영화5|
|---|---|---|---|---|---|
|사용자1|4|NaN|NaN|2|NaN|
|사용자2|NaN|5|NaN|3|1|
|사용자3|NaN|NaN|3|4|4| 
|사용자4|5|2|1|2|NaN|

NaN 칸은 아직 사용자가 평점을 남기지 않은 영화로, NaN으로 작성된 칸을 잠재요인 행렬분해를 통해 예측해보겠다.

In [12]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정. 
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])
num_users, num_items = R.shape
K=3

# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다. 
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
# print(P)
Q = np.random.normal(scale=1./K, size=(num_items, K))
# print(Q)

경사하강법을 통해 P와 Q행렬로 분해가 되었다면, P와 Q행렬의 내적의 곱을 통해 행렬 R에서 NaN으로 이루어진 값들을 채워넣을 수 있다.    
하지만, 예측된 R행렬과 실제 R행렬이 100% 같을 수는 없어서 오차가 생기게 되는데 get_rmse()함수를 통해 오차를 알아보고 오차의 크기를 점차 줄여서 최적화된 예측 R행렬을 얻을 것이다.

In [13]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

행렬분해 기법 중 하나인 SVD(Singular Value Decompositon)방식은 NaN값이 없는 행렬에서 적용이 가능하기 때문에 NaN값이 아닌 요소들을 non_zeros 리스트에 넣어 행렬을 분해할 것이다.    
(이와 같은 이유로 위의 get_rmse()함수에서도 원본행렬 R을 R_non_zeros리스트에 넣은 것이다.)

In [14]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장. 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000
learning_rate=0.01 # learning_rate는 SGD(경사하강법)의 학습률 (편미분된 결과값이 너무 커지는 것을 막기 위함)
r_lambda=0.01 # r_lambda는 L2 Regularizaion rPtn (overfitting을 막기 위함)

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트. 
for step in range(steps):
    for i, j, r in non_zeros:
        # i: 데이터의 x값, j: 데이터의 y값, r: 데이터가 가지고 있는 값(=0이 아닌 값)
        # 실제 값과 예측 값의 차이인 오류 값(=eij) 구함 (np.dot()을 사용하여 내적의 곱 수행)
        eij = r - np.dot(P[i, :], Q[j, :].T) # Q는 전치행렬을 사용해야하기 때문에 .T메소드를 사용하여 전치행렬로 변환한다.
        # Regularization을 반영한 SGD 업데이트 공식 적용 (경사하강법)
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.48767231013696477
### iteration step :  100  rmse :  0.15643403848192472
### iteration step :  150  rmse :  0.07455141311978056
### iteration step :  200  rmse :  0.04325226798579326
### iteration step :  250  rmse :  0.0292483287808792
### iteration step :  300  rmse :  0.022621116143829462
### iteration step :  350  rmse :  0.019493636196525253
### iteration step :  400  rmse :  0.018022719092132912
### iteration step :  450  rmse :  0.017319685953442663
### iteration step :  500  rmse :  0.01697365788757087
### iteration step :  550  rmse :  0.01679680459589556
### iteration step :  600  rmse :  0.016701322901884613
### iteration step :  650  rmse :  0.016644736912476723
### iteration step :  700  rmse :  0.01660591006820994
### iteration step :  750  rmse :  0.016574200475704952
### iteration step :  800  rmse :  0.016544315829216106
### iteration step :  850  rmse :  0.016513751774735172
### it

iteration step이 950일 경우    
즉, 1000이 될수록 원본행렬R과 예측행렬R 사이의 rmse(오차값)이 줄어드는 것을 확인할 수 있다.

In [15]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


### 원본행렬 R
||영화1|영화2|영화3|영화4|영화5|
|---|---|---|---|---|---|
|사용자1|4|NaN|NaN|2|NaN|
|사용자2|NaN|5|NaN|3|1|
|사용자3|NaN|NaN|3|4|4| 
|사용자4|5|2|1|2|NaN|
    
### 예측행렬 pred_matrix
||영화1|영화2|영화3|영화4|영화5|
|---|---|---|---|---|---|
|사용자1|3.991|0.897|1.306|2.002|1.663|
|사용자2|6.696|4.978|0.979|2.981|1.003|
|사용자3|6.677|0.391|2.987|3.977|3.986| 
|사용자4|4.968|2.005|1.006|2.017|1.14|


위의 표를 확인해보면 원본행렬과 예측행렬이 매우 유사하다는 것을 알 수 있다.    
이 결과값을 토대로 사용자1이 영화2에 대해서는 호감도가 별로 없을 것이고, 사용자3이 영화1에 대해 호감도가 높을 것으로 예측되어 영화1을 추천해주는 것이다.